<center> Imports <center>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import numpy as np


 <center> Import check_in data from Yelp dataset <center>

In [ ]:
file = '/Users/Lisa/Downloads/yelp_dataset/yelp_academic_dataset_checkin.json'

In [ ]:
checkins = pd.read_json(file, lines=True)

In [ ]:
checkins

 <center> Since the date column is a long string I'll need to change it to a list in order to leverage date/time for future graphing <center>

In [ ]:
checkins['date'] = checkins['date'].apply(lambda x: x.split(","))

 <center> Leverage dataframe.explode('column_name') to break the list of check-in's down <center>

In [ ]:
checkins_broken_down = checkins.explode('date')

 <center> From my EDA is seems like each business has a unique business_id which gives a breakdown of the dates people checked in to these businesses. I can use that to compare against the weather for those dates. <br/> Now Identify the locations with enough data points to scrap weather data for <center>

 <center> Import Business data from Yelp dataset <center>

In [ ]:
file2 = '/Users/Lisa/Downloads/yelp_dataset/yelp_academic_dataset_business.json'

In [ ]:
businesses = pd.read_json(file2, lines=True)

 <center> Clean up businesses dataframe and elimnate unnecessary columns since its not data I need to focus on <center>

In [ ]:
businesses = businesses.drop(['hours', 'categories', 'attributes', 'longitude', 'latitude', 'stars', 'is_open', 'address', 'postal_code', 'review_count'], axis=1)

 <center> Idnetify a State with enough data points to work with (since there's a lot of entries) <center>

In [ ]:
businesses.groupby('state').count()

<center>It seems like Arizona has the most amount of Yelp Activity <br/> Determine which city to focus on in AZ. <br/> Groupby and sort the counted data<center>

In [ ]:
AZ = businesses.loc[businesses['state'] == 'AZ'].groupby('city').count()

<center>Take the grouped data and sort it<center>

In [ ]:
AZ.sort_values('state', ascending=False)

 <center> Through EDA, it seems like Phoenix has the greatest activity to measure against weather. Will focus on Yelp check-in's there<center>

 <center> Pull Pheonix, AZ from the businesses dataframe <center>

In [ ]:
phx_az_biz = businesses.loc[(businesses['city']=='Phoenix') & (businesses['state']=='AZ')]

 <center> Merge Phoenix, AZ dataframe to checkins_broken_down <center>

In [ ]:
phx_checkins = phx_az_biz.merge(checkins_broken_down, how='left', on='business_id')

 <center> get date/time to a workable data type (not a string!) <center>

<center> Example: data["Date"]= pd.to_datetime(data["Date"]) <center>

In [ ]:
phx_checkins['date'] = pd.to_datetime(phx_checkins['date'])

In [ ]:
phx_checkins['date2.0'] = phx_checkins['date'].dt.date

In [ ]:
phx_checkins = phx_checkins.drop('date', axis=1)

 <center> Groupby and count dates <center>

 <center> Break up phx_checkins by years <center>

In [ ]:
phx_checkins

In [ ]:
# check to see how to access date/time
print(phx_checkins.iloc[0,4].month)
print(phx_checkins.iloc[0,4].day)
print(phx_checkins.iloc[0,4].year)

In [ ]:
# new columns for month, date, year
phx_checkins['Month'] = phx_checkins['date2.0'].apply(lambda x: x.month)

In [ ]:
phx_checkins['Day'] = phx_checkins['date2.0'].apply(lambda x: x.day)

In [ ]:
phx_checkins['Year'] = phx_checkins['date2.0'].apply(lambda x: x.year)

In [ ]:
phx_checkins

 <center> Get a breakdown by year and the total amount of checkins. Next will be to separate them into years <center>

In [ ]:
phx_checkins_years = phx_checkins.groupby('Year').count()
phx_checkins_years

In [ ]:
checkins_2019 = phx_checkins.loc[phx_checkins['Year'] == 2019]
checkins_2019

 <center> Now that I have 2019 checkin data, break it down by months for plotting<center>

In [ ]:
# Get a count of businesses per day
# phx_az_biz = businesses.loc[(businesses['city']=='Phoenix') & (businesses['state']=='AZ')]

In [ ]:
test = checkins_2019.loc[ (checkins_2019['Day'] == 1) & (checkins_2019['Month'] == 1)]
test = test.groupby('business_id').count()
# test = test.reset_index()
# test = test['business_id'].unique()
test

In [ ]:
# Doesn't sound like I'll need to leverage the amount of restaraunts that were being check in to, onlt the total per day
'''
def rest_per_day_jan(df):
    lst = []
    a = np.arange(1,32)
    for date in a:
        total = df.loc[ (df['Day'] == date) & (df['Month'] == 1)]
        total = total.groupby('business_id').count()
        lst.append(len(total))
    return lst

rest_per_day_jan(checkins_2019)
'''

In [ ]:
def create_df(x):
    x = x['Day'].sort_values()
    x = x.to_frame()
    return x

In [ ]:
jan_checkins_2019 = checkins_2019[checkins_2019['Month'] == 1]
jan_hist = create_df(jan_checkins_2019)
jan_hist

In [ ]:
# Each months data points
feb_checkins_2019 = checkins_2019[checkins_2019['Month'] == 2]
# feb_hist = create_df(feb_checkins_2019)
mar_checkins_2019 = checkins_2019[checkins_2019['Month'] == 3]
# mar_hist = create_df(march_checkins_2019)
apr_checkins_2019 = checkins_2019[checkins_2019['Month'] == 4]
# apr_hist = create_df(april_checkins_2019)
may_checkins_2019 = checkins_2019[checkins_2019['Month'] == 5]
# may_hist = create_df(may_checkins_2019)
jun_checkins_2019 = checkins_2019[checkins_2019['Month'] == 6]
# jun_hist = create_df(june_checkins_2019)
jul_checkins_2019 = checkins_2019[checkins_2019['Month'] == 7]
# jul_hist = create_df(july_checkins_2019)
aug_checkins_2019 = checkins_2019[checkins_2019['Month'] == 8]
# aug_hist = create_df(august_checkins_2019)
sep_checkins_2019 = checkins_2019[checkins_2019['Month'] == 9]
# sep_hist = create_df(sept_checkins_2019)
oct_checkins_2019 = checkins_2019[checkins_2019['Month'] == 10]
# oct_hist = create_df(oct_checkins_2019)
nov_checkins_2019 = checkins_2019[checkins_2019['Month'] == 11]
# nov_hist = create_df(nov_checkins_2019)
dec_checkins_2019 = checkins_2019[checkins_2019['Month'] == 12]
# dec_hist = create_df(dec_checkins_2019)

### <center> Test out graphing by month <center>

In [ ]:
x_ticks = jan_checkins_2019['Day'].unique()
x_ticks = np.sort(x_ticks)
x_ticks

In [ ]:
fig, ax = plt.subplots(figsize=(16,8), sharex=True, sharey=True)
ax.hist(x = jan_checkins_2019['Day'], bins=np.arange(1,32)-1, align='left')
#ax.hist(x = jan_checkins_2019['Day'], bins=x_ticks, align='mid')
plt.xticks(x_ticks)
plt.tight_layout()
ax.set_xlabel('Day')
ax.set_ylabel('Total Checkins')
ax.title.set_text("January 2019")
# ax.plot(rest_per_day_jan(checkins_2019),color='r')

In [ ]:
# GET checkins DF cleaned up

In [ ]:
# columns to drop
drop = ['business_id', 'city', 'state', 'date2.0', 'Month', 'Year']

In [ ]:
jan = jan_checkins_2019.groupby('Day').count()
jan = jan.reset_index()
jan.drop(drop, axis=1)

In [ ]:
feb = feb_checkins_2019.groupby('Day').count()
feb = feb.reset_index()
feb.drop(drop, axis=1)

In [ ]:
mar = mar_checkins_2019.groupby('Day').count()
mar = mar.reset_index()
mar.drop(drop, axis=1)

In [ ]:
apr = apr_checkins_2019.groupby('Day').count()
apr = apr.reset_index()
apr.drop(drop, axis=1)

In [ ]:
may = may_checkins_2019.groupby('Day').count()
may = may.reset_index()
may.drop(drop, axis=1)

In [ ]:
jun = jun_checkins_2019.groupby('Day').count()
jun = jun.reset_index()
jun.drop(drop, axis=1)

In [ ]:
# Monthly weather data imports
jan2 = pd.read_csv('/Users/Lisa/capstone_1/jan.csv')
feb2 = pd.read_csv('/Users/Lisa/capstone_1/feb.csv')
mar2 = pd.read_csv('/Users/Lisa/capstone_1/mar.csv')
apr2 = pd.read_csv('/Users/Lisa/capstone_1/apr.csv')
may2 = pd.read_csv('/Users/Lisa/capstone_1/may.csv')
jun2 = pd.read_csv('/Users/Lisa/capstone_1/jun.csv')
# jul2 = pd.read_csv('/Users/Lisa/capstone_1/jul.csv')
# aug2 = pd.read_csv('/Users/Lisa/capstone_1/aug.csv')
# sep2 = pd.read_csv('/Users/Lisa/capstone_1/sep.csv')
# _oct2 = pd.read_csv('/Users/Lisa/capstone_1/oct.csv')
# nov2 = pd.read_csv('/Users/Lisa/capstone_1/nov.csv')
# dec2 = pd.read_csv('/Users/Lisa/capstone_1/dec.csv')

In [ ]:
# jan = jan.merge(jan, how='left', on='Day')
# jan.drop(drop, axis=1)
jan